<a href="https://colab.research.google.com/github/aabchyein/study_data_analystics/blob/main/codes/NLP/healthapp_review_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

refer : https://heytech.tistory.com/401

In [12]:
!pip install python-mecab-ko pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.

In [13]:
import numpy as np

In [2]:
import pandas as pd
df_raw = pd.read_excel('./dataset_raw.xlsx')
df_raw[:2]

,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5


## 전처리

### 결측치 처리

In [3]:
df_raw.info()  # review에 결측치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


In [4]:
df_raw_dropna = df_raw.dropna()
df_raw_dropna.isnull().sum()

app       0
review    0
rating    0
dtype: int64

### 자연어 전처리

#### 특정(상업용) 앱은 리스트에서 제거
- 분석에 의미가 없다고 판단되어 제거

In [5]:
remove_app_list = pd.read_excel('./remove_app_list.xlsx')
remove_app_list[:5]

,app
0,캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1,만보기 - 걸음 계산기
2,딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3,타임캐시 – 돈버는 어플
4,돈버는어플 - 캐시런


In [6]:
remove_app_list.info()  # 106 개의 app은 삭제하려고 함

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app     106 non-null    object
dtypes: object(1)
memory usage: 976.0+ bytes


In [7]:
for remove_app in remove_app_list['app'] :
  try:
    df_removed_app = df_raw_dropna[df_raw_dropna['app'] != remove_app]
  except :
    pass

In [8]:
df_removed_app = df_removed_app[500:2000]  # 106 개의 app이 삭제된 것이 출력됨

### 형태소 적용과 불용어 처리

In [9]:
# 불용어 사전 불러오기
df_stopwords = pd.read_excel('./stopword_list.xlsx')
df_stopwords[:5] # 시리즈

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [10]:
from mecab import MeCab
mecab = MeCab() # 인스턴스화
stopwords = df_stopwords['stopword'].values # -> df_stopwords는 시리즈 형태이기 때문에 값만 넣어주거나 리스트로 만들어줘야 함


# 토크나이징 함수 정의(한 글자 이상 적용, 불용어 삭제)
def tokenizer(raw, stopword=stopwords):
    result_list = list()
    for word in mecab.nouns(raw) :
        if len(word) > 1 and word not in stopword :
            result_list.append(word)  # 리스트에 담아두면 나중에 join을 써서 문자열로 변경하기 쉽다.
    # 일부 경우(for문을 돌리고 난 후의 결과를 한 번 더 돌리는 경우가 있음 -> 토큰 갯수 제한)
    if len(result_list) <= 3 or len(result_list) < 12 :  # 분야별로 다르기는 하나 토큰 갯수가 너무 적거나 많은 경우 일정 범위 이내로 잘라버림.
      result_list
    return ' '.join(result_list) # return str (최종적으로 dataframe 셀에 문자열이 들어갈 수 있게 됨)

In [11]:
mecab.nouns('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

['곳', '운동', '수']

In [12]:
# 형태 분석기 적용된 함수 실행
tokenizer('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

'운동'

In [13]:
df_removed_app['tokenized_review'] = df_removed_app['review'].apply(tokenizer)
df_removed_app[:5]

,app,review,rating,tokenized_review
500,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요ㅎ,5,
501,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,nice app,5,
502,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,다 영어여서 쓰질 못해먹겠어!!!,1,영어
503,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,한글업데이트하면완벽,4,한글 업데이트 완벽
504,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,따라하기는 좋은데 매뉴만 한글이고 자세한 설명은 영어로 되서 보기 힘드네여,5,한글 설명 영어


#### 유사어 대치
- ㄷㄱㄷㅅㄷ 와 같은 댓글이 있는 경우 이런 댓글들을 살리고 싶다면 대치를 먼저 실행하고 형태소 분석을 실행해야함

In [14]:
df_replace_list = pd.read_excel('./replace_list.xlsx')
df_replace_list[:3]

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰


In [15]:
def replace_word(tokenized_review):
  replaced_review = str()
  for i in range(len(df_replace_list['before_replacement'])):
    try:
      # 치환할 단어가 있는 경우에만 데이터 치환 수행
      if df_replace_list['before_replacement'][i] in tokenized_review:
        replaced_review = tokenized_review.replace(df_replace_list['before_replacement'][i], df_replace_list['after_replacement'][i])
    except Exception as e:
      print(f"Error 발생 / 에러명: {e}")
  return replaced_review

In [16]:
replace_word('S헬스 한글 LG V10 업데이트 완벽')

'S헬스 한글 스마트폰 업데이트 완벽'

In [17]:
replace_word('')

''

In [18]:
df_removed_app['replaced_review'] = df_removed_app['tokenized_review'].apply(replace_word)
df_removed_app[:2]

,app,review,rating,tokenized_review,replaced_review
500,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요ㅎ,5,,
501,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,nice app,5,,


In [19]:
df_removed_app.to_csv('./preprocess_reviews.csv')

### 부정과 긍정 댓글 분리

In [20]:
condition_positive = df_removed_app['rating'] >= 3   # 긍정
condition_negative = df_removed_app['rating'] < 3   # 부정

In [21]:
df_review_positive = df_removed_app[condition_positive]
df_review_negative = df_removed_app[condition_negative]

In [22]:
df_review_positive[:3]

,app,review,rating,tokenized_review,replaced_review
500,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,좋아요ㅎ,5,,
501,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,nice app,5,,
503,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,한글업데이트하면완벽,4,한글 업데이트 완벽,


In [23]:
df_review_negative[:3]

,app,review,rating,tokenized_review,replaced_review
502,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,다 영어여서 쓰질 못해먹겠어!!!,1,영어,
505,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,회원탈퇴도 안되고 설명도 재대로 안되있는거같고 다 유료. 잖아요,1,회원 탈퇴 설명 유료,
543,JEFIT - Fit 헬스장 트레이너 피트니스 운동관리 운동일지 트래커 Training,Not install download. . . .,2,,


### 긍정, 부정으로 전처리한 것을 CSV 파일로 저장

In [24]:
df_review_positive.to_csv('./review_positive.csv')
df_review_negative.to_csv('./review_negative.csv')

## LDA 분석